# test GUIDER ROI production code, v2

In [ ]:
from lsst.daf.butler import Butler
from lsst.ts.observatory.control.utils.extras.guider_roi import GuiderROIs  # noqa: F401                                                                                                                  

from astropy.table import Table, vstack
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
repo_name = "LSSTCam"
catalog_dataset = "guider_roi_monster_guide_catalog"
vignetting_dataset = "guider_roi_vignetting_correction"
collection = "guider_roi_data"
groi = GuiderROIs(
                catalog_name=catalog_dataset,
                vignetting_dataset=vignetting_dataset,
                collection=collection,
                repo_name=repo_name,
            )

In [ ]:
ra =  32.27 
dec = -21.29 
skyang = 24.17

In [ ]:
roi_spec, selected_stars = groi.get_guider_rois(
            ra=ra,
            dec=dec,
            sky_angle=skyang,  # degrees                                                                                                                                                                  
            roi_size=400,  # pixels (FIXED right now, should make an argument)                                                                                                                            
            roi_time=200,  # ms (max allowed by ROICommon validation: 5-200)                                                                                                                              
            band="y",  # CHANGE to Y by HAND                                                                                                                                                              
            use_guider=True,
            use_science=False,
            use_wavefront=False,
        )

In [ ]:
selected_stars.columns

In [ ]:
selected_stars[['ccdName','ampNameLL','ampxLL','ampyLL','mag_y','gaia_G','delta_mag']]

In [ ]:
type(selected_stars)

In [ ]:
for key in roi_spec.roi.keys():
    print(key, roi_spec.roi[key])

# get ra,dec,skyang for nights of on-sky images and assess the resulting Guider catalogs

In [ ]:
import sqlite3
import pandas as pd


In [ ]:
data = pd.read_parquet('baseline_v5.0.0_10years_10k.parquet')

In [ ]:
print(data[['fieldRA','fieldDec','filter','rotSkyPos','rotTelPos','band']])

In [ ]:
ras = data['fieldRA'].to_numpy()
decs = data['fieldDec'].to_numpy()
bands = data['filter'].to_numpy()
skyangs = data['rotSkyPos'].to_numpy()

## get stars for all these 10k visits

In [ ]:
tables = []
for i in range(2000):
    ra = ras[i]
    dec = decs[i]
    band = bands[i]
    skyang = skyangs[i]
    roi_spec, selected_stars = groi.get_guider_rois(
            ra=ra,
            dec=dec,
            sky_angle=skyang,  # degrees                                                                                                                                                                  
            roi_size=400,  # pixels (FIXED right now, should make an argument)                                                                                                                            
            roi_time=200,  # ms (max allowed by ROICommon validation: 5-200)                                                                                                                              
            band=band,  # use the band                                                                                                                                                         
            use_guider=True,
            use_science=False,
            use_wavefront=False,
        )
    selected_stars['ivisit'] = i
    selected_stars['nstars_visit'] = len(selected_stars)
    selected_stars['band'] = band
    tables.append(selected_stars)
    
        

In [ ]:
alltable = vstack(tables)

In [ ]:
alltable.columns

# Plots

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.hist(alltable['gaia_G'],bins=100)

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.hist(alltable['nstars_visit'],bins=10,range=(0,10))

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.scatter(np.rad2deg(alltable['coord_ra']),np.rad2deg(alltable['coord_dec']))

In [ ]:
f,ax = plt.subplots(1,1,figsize=(4,8))
_ = ax.scatter(alltable['ampxLL'],alltable['ampyLL'],s=0.1)
ax.set_aspect('equal')
ax.set_xlabel('Amplifier X')
ax.set_ylabel('Amplifier Y')
f.suptitle('Amplifier position for Guider ROI LL')

In [ ]:
from collections import Counter

# Count the unique values in the 'fruit' column
unique_counts = Counter(alltable['ampName'])

# Print the result
unique_counts

In [ ]:
f,ax = plt.subplots(1,1,figsize=(8,8))

from matplotlib.patches import Rectangle

# Define the properties of the rectangle
# (x, y) is the lower-left corner, followed by width and height
x = 0.0
y = 0.0
width = 4072.0
height = 4000.0

# Create the Rectangle patch
rectangle = Rectangle((x, y), width, height,
                      facecolor='white',  # Fill color
                      edgecolor='green',       # Outline color
                      linewidth=1)            # Outline thickness

# Add the rectangle to the axes
ax.add_patch(rectangle)

_ = ax.scatter(alltable['ccdxLL'],alltable['ccdyLL'],s=0.1)
ax.set_aspect('equal')
ax.set_xlabel('CCD X')
ax.set_ylabel('CCD Y')
f.suptitle('CCD position for Guider ROI LL')

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.hist(alltable['ampxLL'],bins=100)

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.hist(alltable['ampyLL'],bins=100)

In [ ]:
f,ax = plt.subplots(1,1,figsize=(8,8))


# Define the properties of the rectangle
# (x, y) is the lower-left corner, followed by width and height
x = 0.0
y = 0.0
width = 4072.0
height = 4000.0

# Create the Rectangle patch
rectangle = Rectangle((x, y), width, height,
                      facecolor='white',  # Fill color
                      edgecolor='blue',       # Outline color
                      linewidth=1)            # Outline thickness

# Add the rectangle to the axes
ax.add_patch(rectangle)
_ = ax.scatter(alltable['ccdx'],alltable['ccdy'],s=0.1)
ax.set_aspect('equal')
ax.set_xlabel('CCD X')
ax.set_ylabel('CCD Y')
f.suptitle('CCD position for Guider Stars')

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.hist(alltable['ccdx'],bins=100,histtype='step')
ax.set_xlabel('CCD X')
f.suptitle('CCD position for Guider Stars')

In [ ]:
f,ax = plt.subplots(1,1)
_ = ax.hist(alltable['ccdy'],bins=100,histtype='step')
ax.set_xlabel('CCD Y')
f.suptitle('CCD position for Guider Stars')

# tests with LSSTCam

In [ ]:
from lsst.afw import cameraGeom
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
camera = LsstCam.getCamera()

In [ ]:
ccdx = 0
ccdy = 0
for detector in camera:
    if detector.getType() == cameraGeom.DetectorType.GUIDER:
        print(detector.getName(),detector.getPhysicalType())
        lct = LsstCameraTransforms(camera, detector.getName())

        # check LL of the CCDs
        ccdx = 0
        ccdy = 0
        ampNameLL, ampxLL, ampyLL = lct.ccdPixelToAmpPixel(ccdx, ccdy)
        print('CCD LL ',ampNameLL,ampxLL,ampyLL)

        # check UR of the CCDs
        ccdx = detector.getBBox().getMaxX()
        ccdy = detector.getBBox().getMaxY()
        ampNameLL, ampxLL, ampyLL = lct.ccdPixelToAmpPixel(ccdx, ccdy)
        print('CCD UR ',ampNameLL,ampxLL,ampyLL)
    elif detector.getType() == cameraGeom.DetectorType.WAVEFRONT:
        print(detector.getName(),detector.getPhysicalType())
        lct = LsstCameraTransforms(camera, detector.getName())

        # check LL of the CCDs
        ccdx = 0
        ccdy = 0
        ampNameLL, ampxLL, ampyLL = lct.ccdPixelToAmpPixel(ccdx, ccdy)
        print('CCD LL ',ampNameLL,ampxLL,ampyLL)

        # check UR of the CCDs
        ccdx = detector.getBBox().getMaxX()
        ccdy = detector.getBBox().getMaxY()
        ampNameLL, ampxLL, ampyLL = lct.ccdPixelToAmpPixel(ccdx, ccdy)
        print('CCD UR ',ampNameLL,ampxLL,ampyLL)
    
    else:
        print(detector.getName(),detector.getPhysicalType())
